# Capstone: Pike Place Fish Market AI Buyer

An end-to-end agentic system that composes **all 14 patterns** from Chapters 2–8 plus new patterns into a single realistic pipeline: buying fresh fish from Seattle’s Pike Place Market.

```
Customer Request
      │
      ▼
[Coordinator] ── Ch 2 Routing ── extracts fish_type, qty, budget, deadline
      │
      ▼
[ParallelVendorChecker] ── Ch 3 ── queries 5 vendors concurrently
      │
      ▼
[QualityInspector] ── Ch 6 Reflection ── rejects low-freshness options
      │
      ▼
[SmartBuyer] ── Ch 8a ReAct + Learning ── picks best using past history
      │
      ▼
[Negotiator ↔ VendorAgent] ── A2A Protocol ── structured price negotiation
      │
      ▼
[GoalMonitor] ── Goal Tracking ── checks budget/freshness/deadline
      │
      ▼
[Human Approval Gate] ── HITL ── "Approve $140 for 5lb King Salmon?"
      │
      ▼
[Execute Purchase] ── Exception Handling ── retry on failure, fallback
      │
      ▼
[Record & Learn] ── Memory + Learning ── save outcome, update scores
```

### All 14 Patterns Mapped

| Pattern | Chapter | How It Appears Here |
|---------|---------|--------------------|
| Routing | Ch 2 | Coordinator routes by fish type |
| Parallelization | Ch 3 | 5 vendor checkers query concurrently |
| Tool Use | Ch 5 | query_catalog, check_freshness, estimate_shipping, negotiate_price |
| Reflection | Ch 6 | QualityInspector rejects low-freshness picks |
| Multi-Agent Collaboration | Ch 7 | 7 agents with specialized roles |
| Sequential Pipeline | Ch 7 | Coordinator → Buyer → Inspector → Negotiator |
| AgentTool | Ch 7 | VendorAgent wrapped as tool for Negotiator |
| ReAct | Ch 8a | SmartBuyer explores options iteratively |
| Plan-and-Execute | Ch 8b | CapstoneOrchestrator with replanning on failure |
| Memory Management | New | PurchaseMemory — stores history, vendor scores |
| Learning & Adaptation | New | LearningEngine — best vendor, price prediction |
| MCP Server | New | MockMCPVendorServer — simulated external tool server |
| Agent-to-Agent (A2A) | New | NegotiationProtocol — structured offer/counter-offer |
| Goal Setting & Monitoring | New | GoalTracker — budget, freshness, deadline |
| Exception Handling | New | ExceptionHandler — out_of_stock, price_spike recovery |
| Human-in-the-Loop | New | Approval gate before purchase execution |

### The Scenario

You’re ordering fresh fish from Seattle’s Pike Place Market for a dinner party. Five vendors compete on price, freshness, and reliability. Your AI system discovers options, negotiates prices, handles surprises, learns from past purchases, and asks you before spending your money.

In [17]:
import os
import json
import time
import nest_asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any

nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()
assert os.environ.get("GOOGLE_API_KEY"), "Set GOOGLE_API_KEY first"
print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

Google API Key set: True


In [18]:
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool
from google.genai import types
from google.genai import types as genai_types

# ── ADK-native retry config (Ch 8b lesson) ────────────────
RETRY_CONFIG = genai_types.GenerateContentConfig(
    http_options=genai_types.HttpOptions(
        retry_options=genai_types.HttpRetryOptions(
            initial_delay=2,
            attempts=3,
        ),
    ),
)

MODEL = "gemini-2.0-flash"
print(f"Using model: {MODEL} with auto-retry on 429s")

Using model: gemini-2.0-flash with auto-retry on 429s


---
## The Pike Place Market Ecosystem

Five vendors, each with different strengths:

| Vendor | Specialty | Price Range | Freshness | Reliability |
|--------|-----------|-------------|-----------|-------------|
| **Wild Salmon Co** | Premium King Salmon | $28-32/lb | 9-10/10 | 95% on-time |
| **Atlantic Fishery** | Farmed Salmon | $16-20/lb | 6-7/10 | 90% on-time |
| **Halibut House** | Pacific Halibut | $32-38/lb | 8-9/10 | 85% on-time |
| **Tuna King** | Yellowfin Tuna | $20-25/lb | 7-8/10 | 92% on-time |
| **Crab Cart** | Dungeness Crab | $14-18/lb | 8-9/10 | 88% on-time |

All data is **deterministic mock** (same principle as Ch 6 and Ch 8a) so the pipeline produces the same results every run.

In [19]:
# ── Pike Place Vendor Data ───────────────────────────────
VENDORS = {
    "Wild Salmon Co": {
        "specialty": ["King Salmon", "Sockeye Salmon", "Coho Salmon"],
        "inventory": {"King Salmon": 25, "Sockeye Salmon": 40, "Coho Salmon": 15},
        "price_per_lb": {"King Salmon": 28, "Sockeye Salmon": 22, "Coho Salmon": 19},
        "freshness_score": {"King Salmon": 9, "Sockeye Salmon": 10, "Coho Salmon": 8},
        "reliability": 0.95,
        "min_margin": 0.20,  # won't sell below cost + 20%
        "cost_basis": {"King Salmon": 18, "Sockeye Salmon": 14, "Coho Salmon": 12},
    },
    "Atlantic Fishery": {
        "specialty": ["King Salmon", "Atlantic Salmon"],
        "inventory": {"King Salmon": 50, "Atlantic Salmon": 100},
        "price_per_lb": {"King Salmon": 18, "Atlantic Salmon": 14},
        "freshness_score": {"King Salmon": 6, "Atlantic Salmon": 7},
        "reliability": 0.90,
        "min_margin": 0.15,
        "cost_basis": {"King Salmon": 12, "Atlantic Salmon": 9},
    },
    "Halibut House": {
        "specialty": ["Pacific Halibut", "Lingcod"],
        "inventory": {"Pacific Halibut": 12, "Lingcod": 20},
        "price_per_lb": {"Pacific Halibut": 35, "Lingcod": 24},
        "freshness_score": {"Pacific Halibut": 9, "Lingcod": 8},
        "reliability": 0.85,
        "min_margin": 0.25,
        "cost_basis": {"Pacific Halibut": 22, "Lingcod": 15},
    },
    "Tuna King": {
        "specialty": ["Yellowfin Tuna", "Albacore Tuna"],
        "inventory": {"Yellowfin Tuna": 30, "Albacore Tuna": 45},
        "price_per_lb": {"Yellowfin Tuna": 22, "Albacore Tuna": 16},
        "freshness_score": {"Yellowfin Tuna": 8, "Albacore Tuna": 7},
        "reliability": 0.92,
        "min_margin": 0.18,
        "cost_basis": {"Yellowfin Tuna": 14, "Albacore Tuna": 10},
    },
    "Crab Cart": {
        "specialty": ["Dungeness Crab", "King Crab Legs"],
        "inventory": {"Dungeness Crab": 35, "King Crab Legs": 8},
        "price_per_lb": {"Dungeness Crab": 16, "King Crab Legs": 45},
        "freshness_score": {"Dungeness Crab": 9, "King Crab Legs": 7},
        "reliability": 0.88,
        "min_margin": 0.22,
        "cost_basis": {"Dungeness Crab": 10, "King Crab Legs": 30},
    },
}

print(f"Pike Place Market: {len(VENDORS)} vendors loaded")
for name, v in VENDORS.items():
    print(f"  {name}: {', '.join(v['specialty'])}")

Pike Place Market: 5 vendors loaded
  Wild Salmon Co: King Salmon, Sockeye Salmon, Coho Salmon
  Atlantic Fishery: King Salmon, Atlantic Salmon
  Halibut House: Pacific Halibut, Lingcod
  Tuna King: Yellowfin Tuna, Albacore Tuna
  Crab Cart: Dungeness Crab, King Crab Legs


---
## Infrastructure Classes

These are the **new patterns** from the chapters you just read — memory, learning, goals, exceptions, and agent-to-agent protocol. Each is a plain Python class that agents use via tools.

| Class | Pattern | Purpose |
|-------|---------|---------|
| `PurchaseMemory` | Memory Management | Stores past purchases, vendor reliability scores |
| `LearningEngine` | Learning & Adaptation | Finds best vendors, predicts prices from history |
| `GoalTracker` | Goal Setting & Monitoring | Tracks budget, freshness, deadline constraints |
| `ExceptionHandler` | Exception Handling | Logs failures, recommends recovery strategies |
| `NegotiationProtocol` | Agent-to-Agent (A2A) | Structured offer/counter-offer between buyer and vendor |

In [20]:
# ── Memory Management ─────────────────────────────────
# Persists across scenarios. The system gets smarter with each purchase.

class PurchaseMemory:
    """Stores past purchases and vendor reliability scores."""

    def __init__(self):
        self.history: List[Dict] = []
        self.vendor_scores: Dict[str, List[float]] = {}  # vendor -> [score1, score2, ...]

    def add_purchase(self, vendor: str, fish_type: str, qty: float,
                     price_per_lb: float, freshness: int, on_time: bool):
        record = {
            "vendor": vendor, "fish_type": fish_type, "qty": qty,
            "price_per_lb": price_per_lb, "total_cost": qty * price_per_lb,
            "freshness": freshness, "on_time": on_time,
            "timestamp": datetime.now().isoformat(),
        }
        self.history.append(record)
        # Update vendor reliability
        score = (freshness / 10) * (1.0 if on_time else 0.5)
        self.vendor_scores.setdefault(vendor, []).append(score)
        return record

    def get_vendor_reliability(self, vendor: str) -> float:
        scores = self.vendor_scores.get(vendor, [])
        return sum(scores) / len(scores) if scores else 0.5  # default unknown

    def get_history_for(self, fish_type: str) -> List[Dict]:
        return [h for h in self.history if h["fish_type"] == fish_type]

    def summary(self) -> str:
        lines = [f"Purchase history: {len(self.history)} transactions"]
        for vendor, scores in self.vendor_scores.items():
            avg = sum(scores) / len(scores)
            lines.append(f"  {vendor}: {len(scores)} purchases, reliability {avg:.2f}")
        return "\n".join(lines)


# Global memory instance — persists across all phases
memory = PurchaseMemory()
print("PurchaseMemory initialized (empty)")

PurchaseMemory initialized (empty)


In [21]:
# ── Learning & Adaptation ─────────────────────────────
# Analyzes purchase history to make smarter decisions.

class LearningEngine:
    """Learns from past purchases to improve future decisions."""

    def __init__(self, mem: PurchaseMemory):
        self.memory = mem

    def best_vendor_for(self, fish_type: str) -> Optional[str]:
        """Which vendor delivered best quality/price ratio for this fish?"""
        history = self.memory.get_history_for(fish_type)
        if not history:
            return None
        # Score = freshness / price_per_lb (higher is better)
        vendor_scores = {}
        for h in history:
            v = h["vendor"]
            score = h["freshness"] / max(h["price_per_lb"], 1)
            vendor_scores.setdefault(v, []).append(score)
        avg_scores = {v: sum(s)/len(s) for v, s in vendor_scores.items()}
        return max(avg_scores, key=avg_scores.get)

    def avg_price(self, fish_type: str) -> Optional[float]:
        """What's the average price we've paid for this fish?"""
        history = self.memory.get_history_for(fish_type)
        if not history:
            return None
        return sum(h["price_per_lb"] for h in history) / len(history)

    def get_insights(self, fish_type: str) -> str:
        """Return a summary of what we've learned about this fish."""
        best = self.best_vendor_for(fish_type)
        avg = self.avg_price(fish_type)
        history = self.memory.get_history_for(fish_type)
        if not history:
            return f"No purchase history for {fish_type}. This is our first time buying it."
        return (
            f"Learning insights for {fish_type}:\n"
            f"  Past purchases: {len(history)}\n"
            f"  Best vendor: {best}\n"
            f"  Average price: ${avg:.2f}/lb\n"
            f"  Best freshness seen: {max(h['freshness'] for h in history)}/10"
        )


learning = LearningEngine(memory)
print("LearningEngine initialized (linked to PurchaseMemory)")

LearningEngine initialized (linked to PurchaseMemory)


In [22]:
# ── Goal Setting & Monitoring ────────────────────────
# Tracks constraints: budget, freshness, delivery deadline.

class GoalTracker:
    """Monitors purchase goals and flags violations."""

    def __init__(self, budget: float, min_freshness: int, delivery_by: str):
        self.budget = budget
        self.min_freshness = min_freshness  # minimum freshness score (1-10)
        self.delivery_by = delivery_by
        self.spent = 0.0

    def check(self, price: float, freshness: int, delivery_days: int) -> Dict[str, str]:
        """Check all goals. Returns {goal: ON_TRACK|AT_RISK|VIOLATED}."""
        status = {}

        # Budget check
        if self.spent + price > self.budget:
            status["budget"] = "VIOLATED"
        elif self.spent + price > self.budget * 0.8:
            status["budget"] = "AT_RISK"
        else:
            status["budget"] = "ON_TRACK"

        # Freshness check
        if freshness < self.min_freshness:
            status["freshness"] = "VIOLATED"
        elif freshness == self.min_freshness:
            status["freshness"] = "AT_RISK"
        else:
            status["freshness"] = "ON_TRACK"

        # Delivery check (simplified: delivery_days <= days until deadline)
        if delivery_days > 3:
            status["delivery"] = "VIOLATED"
        elif delivery_days > 2:
            status["delivery"] = "AT_RISK"
        else:
            status["delivery"] = "ON_TRACK"

        return status

    def report(self, price: float, freshness: int, delivery_days: int) -> str:
        status = self.check(price, freshness, delivery_days)
        lines = [f"Goal Status (budget: ${self.budget}, min freshness: {self.min_freshness}, deliver by: {self.delivery_by}):"]
        for goal, state in status.items():
            icon = {"ON_TRACK": "✅", "AT_RISK": "⚠️", "VIOLATED": "❌"}[state]
            lines.append(f"  {icon} {goal}: {state}")
        return "\n".join(lines)


print("GoalTracker ready")

GoalTracker ready


In [23]:
# ── Exception Handling & Recovery ─────────────────────
# Logs failures and recommends recovery strategies.

class ExceptionHandler:
    """Tracks exceptions and provides recovery strategies."""

    RECOVERY_STRATEGIES = {
        "OUT_OF_STOCK": "Try next-best vendor from learning insights",
        "PRICE_SPIKE": "Negotiate harder or switch to similar fish type",
        "SHIPPING_DELAY": "Find vendor with local pickup or faster shipping",
        "VENDOR_OFFLINE": "Skip vendor, use backup from parallel check",
        "QUALITY_REJECTED": "Inspector rejected — try different vendor for same fish",
    }

    def __init__(self):
        self.log: List[Dict] = []

    def record(self, exc_type: str, vendor: str, detail: str) -> str:
        entry = {
            "type": exc_type, "vendor": vendor,
            "detail": detail, "timestamp": datetime.now().isoformat(),
            "recovery": self.RECOVERY_STRATEGIES.get(exc_type, "Escalate to human"),
        }
        self.log.append(entry)
        return f"Exception logged: {exc_type} at {vendor}. Recovery: {entry['recovery']}"

    def get_recovery(self, exc_type: str) -> str:
        return self.RECOVERY_STRATEGIES.get(exc_type, "Escalate to human")

    def summary(self) -> str:
        if not self.log:
            return "No exceptions recorded."
        lines = [f"Exception log: {len(self.log)} events"]
        for e in self.log:
            lines.append(f"  [{e['type']}] {e['vendor']}: {e['detail']}")
        return "\n".join(lines)


exceptions = ExceptionHandler()
print("ExceptionHandler ready")

ExceptionHandler ready


In [24]:
# ── Agent-to-Agent (A2A) Negotiation Protocol ────────
# Structured message format for buyer ↔ vendor negotiation.

class NegotiationProtocol:
    """Structured protocol for inter-agent price negotiation."""

    def __init__(self):
        self.rounds: List[Dict] = []

    def create_offer(self, buyer: str, vendor: str,
                     fish_type: str, qty: float, offer_price: float) -> Dict:
        """Buyer creates a structured offer."""
        msg = {
            "protocol": "pike_place_a2a_v1",
            "type": "OFFER",
            "from": buyer, "to": vendor,
            "fish_type": fish_type, "qty_lbs": qty,
            "offer_price_per_lb": offer_price,
            "round": len(self.rounds) + 1,
        }
        self.rounds.append(msg)
        return msg

    def create_counter(self, vendor: str, buyer: str,
                       counter_price: float, available_qty: float) -> Dict:
        """Vendor creates a counter-offer."""
        msg = {
            "protocol": "pike_place_a2a_v1",
            "type": "COUNTER_OFFER",
            "from": vendor, "to": buyer,
            "counter_price_per_lb": counter_price,
            "available_qty": available_qty,
            "round": len(self.rounds) + 1,
        }
        self.rounds.append(msg)
        return msg

    def accept(self, who: str, final_price: float, qty: float) -> Dict:
        """Either party accepts the deal."""
        msg = {
            "protocol": "pike_place_a2a_v1",
            "type": "ACCEPTED",
            "from": who,
            "final_price_per_lb": final_price,
            "final_qty": qty,
            "round": len(self.rounds) + 1,
        }
        self.rounds.append(msg)
        return msg

    def reject(self, who: str, reason: str) -> Dict:
        msg = {
            "protocol": "pike_place_a2a_v1",
            "type": "REJECTED",
            "from": who, "reason": reason,
            "round": len(self.rounds) + 1,
        }
        self.rounds.append(msg)
        return msg

    def trace(self) -> str:
        """Pretty-print the negotiation trace."""
        lines = [f"Negotiation trace ({len(self.rounds)} rounds):"]
        for r in self.rounds:
            lines.append(f"  Round {r['round']}: {r['type']} from {r['from']}")
            if r['type'] == 'OFFER':
                lines.append(f"    {r['qty_lbs']}lb {r['fish_type']} @ ${r['offer_price_per_lb']}/lb")
            elif r['type'] == 'COUNTER_OFFER':
                lines.append(f"    Counter: ${r['counter_price_per_lb']}/lb, {r['available_qty']}lb available")
            elif r['type'] == 'ACCEPTED':
                lines.append(f"    Deal: {r['final_qty']}lb @ ${r['final_price_per_lb']}/lb")
            elif r['type'] == 'REJECTED':
                lines.append(f"    Reason: {r['reason']}")
        return "\n".join(lines)


print("NegotiationProtocol ready")
print("\nAll infrastructure classes initialized:")
print("  PurchaseMemory, LearningEngine, GoalTracker, ExceptionHandler, NegotiationProtocol")

NegotiationProtocol ready

All infrastructure classes initialized:
  PurchaseMemory, LearningEngine, GoalTracker, ExceptionHandler, NegotiationProtocol


---
## Mock Tools & MCP Server

These tools simulate what would be live APIs in production. The MCP server pattern wraps vendor-specific operations behind a standard interface — any vendor can be queried the same way.

| Tool | Purpose | MCP? | Used By |
|------|---------|------|---------|
| `query_vendor_catalog` | Get all vendors carrying a fish type | Yes | Buyer |
| `check_freshness` | Get freshness details for a specific vendor+fish | Yes | Inspector |
| `estimate_shipping` | Get shipping cost and delivery time | Yes | Buyer, GoalMonitor |
| `negotiate_price` | Propose a price to a vendor (A2A) | Yes | Negotiator |
| `get_learning_insights` | Query past purchase history | No | SmartBuyer |
| `record_purchase` | Save completed purchase to memory | No | Orchestrator |
| `check_goals` | Check all goal constraints | No | GoalMonitor |

In [25]:
# ── MCP Vendor Server Simulator ──────────────────────
# In production, each vendor would host their own MCP server.
# We simulate it as a class that ADK tools call into.

class MockMCPVendorServer:
    """Simulates an MCP tool server for a Pike Place vendor.
    
    In production, this would be a real MCP server endpoint.
    The ADK FunctionTool wraps it so agents call it like any other tool.
    """

    def __init__(self, vendor_name: str, vendor_data: Dict,
                 fault_injection: Optional[str] = None):
        self.vendor_name = vendor_name
        self.data = vendor_data
        self.fault = fault_injection  # "OUT_OF_STOCK", "PRICE_SPIKE", etc.

    def get_inventory(self, fish_type: str) -> Dict:
        """MCP operation: get_inventory"""
        if self.fault == "VENDOR_OFFLINE":
            raise ConnectionError(f"{self.vendor_name} is offline")
        if self.fault == "OUT_OF_STOCK" and fish_type in self.data["inventory"]:
            return {"vendor": self.vendor_name, "fish_type": fish_type,
                    "available_lbs": 0, "status": "OUT_OF_STOCK"}

        if fish_type not in self.data["inventory"]:
            return {"vendor": self.vendor_name, "fish_type": fish_type,
                    "available_lbs": 0, "status": "NOT_CARRIED"}

        price = self.data["price_per_lb"][fish_type]
        if self.fault == "PRICE_SPIKE":
            price = int(price * 1.5)  # 50% price spike

        return {
            "vendor": self.vendor_name,
            "fish_type": fish_type,
            "available_lbs": self.data["inventory"][fish_type],
            "price_per_lb": price,
            "freshness_score": self.data["freshness_score"][fish_type],
            "reliability": self.data["reliability"],
            "status": "AVAILABLE",
        }

    def negotiate(self, fish_type: str, qty: float, offer_price: float) -> Dict:
        """MCP operation: negotiate_price"""
        if fish_type not in self.data["cost_basis"]:
            return {"accepted": False, "reason": "Fish not carried"}

        cost = self.data["cost_basis"][fish_type]
        min_price = cost * (1 + self.data["min_margin"])

        if offer_price >= self.data["price_per_lb"][fish_type]:
            # Buyer offered list price or higher — instant accept
            return {"accepted": True, "final_price": self.data["price_per_lb"][fish_type], "qty": qty}
        elif offer_price >= min_price:
            # Offer above minimum margin — counter at midpoint
            counter = round((offer_price + self.data["price_per_lb"][fish_type]) / 2, 2)
            return {"accepted": False, "counter_price": counter, "available_qty": qty}
        else:
            # Below minimum — reject
            return {"accepted": False, "reason": f"Offer ${offer_price}/lb is below our minimum",
                    "counter_price": min_price, "available_qty": qty}


# Create MCP servers for all vendors (no faults by default)
mcp_servers = {
    name: MockMCPVendorServer(name, data)
    for name, data in VENDORS.items()
}
print(f"MCP servers created for {len(mcp_servers)} vendors")

MCP servers created for 5 vendors


In [26]:
# ── ADK Tool Functions (wrap MCP servers) ─────────────
# Each tool calls into the MCP server layer.
# Agents see these as normal FunctionTools.

def query_vendor_catalog(fish_type: str) -> str:
    """Query all Pike Place vendors for a specific fish type.
    Returns availability, price, and freshness from each vendor."""
    results = []
    for name, server in mcp_servers.items():
        try:
            inv = server.get_inventory(fish_type)
            if inv["status"] == "AVAILABLE":
                results.append(inv)
            elif inv["status"] == "OUT_OF_STOCK":
                results.append(inv)
        except ConnectionError as e:
            exceptions.record("VENDOR_OFFLINE", name, str(e))
    if not results:
        return f"No vendors carry {fish_type} at Pike Place Market."
    # Sort by freshness desc, then price asc
    available = [r for r in results if r["status"] == "AVAILABLE"]
    available.sort(key=lambda x: (-x["freshness_score"], x["price_per_lb"]))
    return json.dumps(available, indent=2)


def check_freshness(vendor_name: str, fish_type: str) -> str:
    """Check detailed freshness info for a specific vendor and fish type."""
    server = mcp_servers.get(vendor_name)
    if not server:
        return f"Unknown vendor: {vendor_name}"
    try:
        inv = server.get_inventory(fish_type)
        if inv["status"] != "AVAILABLE":
            return f"{vendor_name} does not have {fish_type} ({inv['status']})"
        freshness = inv["freshness_score"]
        quality = "EXCELLENT" if freshness >= 9 else "GOOD" if freshness >= 7 else "FAIR" if freshness >= 5 else "POOR"
        return (f"{vendor_name} {fish_type}: freshness {freshness}/10 ({quality}). "
                f"Suitable for sashimi: {'Yes' if freshness >= 8 else 'No'}")
    except ConnectionError:
        return f"{vendor_name} is offline"


def estimate_shipping(vendor_name: str, qty_lbs: float) -> str:
    """Estimate shipping cost and delivery time from a vendor."""
    # Mock: shipping cost = $5 base + $1/lb, delivery = 1-2 days
    cost = 5 + qty_lbs * 1.0
    days = 1 if vendor_name in ["Wild Salmon Co", "Atlantic Fishery"] else 2
    return json.dumps({
        "vendor": vendor_name, "shipping_cost": cost,
        "delivery_days": days, "method": "overnight cold pack",
    })


def negotiate_price(vendor_name: str, fish_type: str,
                    qty_lbs: float, offer_price_per_lb: float) -> str:
    """Propose a price to a vendor. Returns acceptance, counter-offer, or rejection."""
    server = mcp_servers.get(vendor_name)
    if not server:
        return f"Unknown vendor: {vendor_name}"
    result = server.negotiate(fish_type, qty_lbs, offer_price_per_lb)
    return json.dumps(result)


def get_learning_insights(fish_type: str) -> str:
    """Get insights from past purchases about this fish type."""
    return learning.get_insights(fish_type)


def record_purchase(vendor_name: str, fish_type: str, qty_lbs: float,
                    price_per_lb: float, freshness_score: int) -> str:
    """Record a completed purchase to memory for future learning."""
    record = memory.add_purchase(vendor_name, fish_type, qty_lbs,
                                 price_per_lb, freshness_score, on_time=True)
    return f"Purchase recorded: {qty_lbs}lb {fish_type} from {vendor_name} @ ${price_per_lb}/lb"


def check_goals(total_price: float, freshness_score: int,
                delivery_days: int) -> str:
    """Check all purchase goals (budget, freshness, delivery)."""
    # Will be called with the current GoalTracker instance
    return "Goal check requires active GoalTracker — see orchestrator."


print("Tools defined:")
for fn in [query_vendor_catalog, check_freshness, estimate_shipping,
           negotiate_price, get_learning_insights, record_purchase, check_goals]:
    print(f"  {fn.__name__}")

Tools defined:
  query_vendor_catalog
  check_freshness
  estimate_shipping
  negotiate_price
  get_learning_insights
  record_purchase
  check_goals


---
## Phase 1: Discovery

**Patterns: Routing (Ch 2) + Parallelization (Ch 3) + Tool Use (Ch 5) + Reflection (Ch 6)**

The customer says what they want. The system:
1. **Coordinator** extracts intent (fish type, qty, budget, deadline)
2. **Buyer** queries all vendors via MCP tools (parallel conceptually, sequential in mock)
3. **Inspector** reviews options against freshness requirements

```
Customer: "5 lbs King Salmon for Saturday dinner, budget $150, sashimi-grade"
         │
         ▼
    [Coordinator] ── extracts: King Salmon, 5lb, $150, min freshness 8
         │
         ▼
    [Buyer] ── queries 5 vendors via MCP ── ranks by freshness+price
         │
         ▼
    [Inspector] ── rejects freshness < 8 ── returns approved list
```

In [27]:
# ── Phase 1 Agents ──────────────────────────────────

buyer_agent = LlmAgent(
    name="BuyerAgent",
    model=MODEL,
    instruction="""
You are a fish buyer at Pike Place Market. Given a customer request,
use query_vendor_catalog to find all vendors with the requested fish.

Then for the top options, use check_freshness to verify quality.
Also use estimate_shipping to check delivery feasibility.

Before searching, call get_learning_insights to check if we have
past purchase data that can guide vendor selection.

Return a ranked list of options with:
- Vendor name, price/lb, freshness score, shipping cost, delivery days
- Recommendation: which vendor is best and why
""",
    tools=[
        FunctionTool(query_vendor_catalog),
        FunctionTool(check_freshness),
        FunctionTool(estimate_shipping),
        FunctionTool(get_learning_insights),
    ],
    generate_content_config=RETRY_CONFIG,
)

inspector_agent = LlmAgent(
    name="QualityInspector",
    model=MODEL,
    instruction="""
You are a quality inspector for Pike Place Market purchases.
Review the buyer's vendor recommendations and check freshness
for each option using the check_freshness tool.

REJECT any option where:
- Freshness score is below 7 (for cooking) or below 8 (if customer wants sashimi/raw)
- Vendor reliability is below 85%

Return only APPROVED options with your quality assessment.
If no options pass inspection, say so clearly.
""",
    tools=[FunctionTool(check_freshness)],
    generate_content_config=RETRY_CONFIG,
)

print("Phase 1 agents ready: BuyerAgent, QualityInspector")

Phase 1 agents ready: BuyerAgent, QualityInspector


In [28]:
# ── Phase 1: Run Discovery Pipeline ──────────────────

async def run_discovery(customer_request: str) -> str:
    """Phase 1: Discover and quality-check vendor options."""
    print(f"{'='*60}")
    print(f"CUSTOMER: {customer_request}")
    print(f"{'='*60}")

    session_service = InMemorySessionService()

    # Step 1: Buyer discovers options
    print("\n[BUYER] Searching vendors...")
    buyer_runner = Runner(agent=buyer_agent, app_name="pike_place",
                          session_service=session_service)
    session = await session_service.create_session(
        app_name="pike_place", user_id="customer")

    content = types.Content(role="user",
        parts=[types.Part.from_text(text=customer_request)])

    buyer_response = ""
    tool_calls = 0
    async for event in buyer_runner.run_async(
        user_id="customer", session_id=session.id, new_message=content):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, "function_call") and part.function_call:
                    tool_calls += 1
                    print(f"  [{tool_calls}] Tool: {part.function_call.name}({dict(part.function_call.args)})")
                elif hasattr(part, "function_response") and part.function_response:
                    result = str(part.function_response.response)[:150]
                    print(f"      Result: {result}...")
                elif hasattr(part, "text") and part.text and part.text.strip():
                    buyer_response += part.text
        if event.is_final_response():
            # Also try to get text from the final event's content
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if hasattr(part, "text") and part.text and part.text.strip():
                        if part.text not in buyer_response:
                            buyer_response += part.text
            break

    print(f"\n[BUYER] Found options ({tool_calls} tool calls)")
    if buyer_response:
        print(f"\n[BUYER RESPONSE]\n{buyer_response[:500]}")

    # Step 2: Inspector quality-checks
    print("\n[INSPECTOR] Reviewing quality...")
    inspector_runner = Runner(agent=inspector_agent, app_name="pike_place",
                              session_service=session_service)
    inspect_session = await session_service.create_session(
        app_name="pike_place", user_id="inspector")

    inspect_msg = types.Content(role="user",
        parts=[types.Part.from_text(
            text=f"Review these vendor options for the customer who wants: {customer_request}\n\n"
                 f"Buyer's findings:\n{buyer_response}")])

    inspector_response = ""
    async for event in inspector_runner.run_async(
        user_id="inspector", session_id=inspect_session.id, new_message=inspect_msg):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, "function_call") and part.function_call:
                    print(f"  Inspector checking: {part.function_call.name}({dict(part.function_call.args)})")
                elif hasattr(part, "text") and part.text and part.text.strip():
                    inspector_response += part.text
        if event.is_final_response():
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if hasattr(part, "text") and part.text and part.text.strip():
                        if part.text not in inspector_response:
                            inspector_response += part.text
            break

    print(f"\n[INSPECTOR] Quality review complete")
    print(f"\n{'='*60}")
    print(f"APPROVED OPTIONS:\n{inspector_response[:500] if inspector_response else '(No text response — inspector may have only used tool calls)'}")
    print(f"{'='*60}")
    return inspector_response


print("Discovery pipeline ready: run_discovery()")

Discovery pipeline ready: run_discovery()


In [29]:
# ── Run Phase 1 ─────────────────────────────────────

discovery_result = await run_discovery(
    "I need 5 lbs of fresh King Salmon for a dinner party this Saturday. "
    "Budget is $150. It needs to be super fresh — I'm serving it as sashimi."
)

CUSTOMER: I need 5 lbs of fresh King Salmon for a dinner party this Saturday. Budget is $150. It needs to be super fresh — I'm serving it as sashimi.

[BUYER] Searching vendors...
  [1] Tool: get_learning_insights({'fish_type': 'King Salmon'})
      Result: {'result': 'No purchase history for King Salmon. This is our first time buying it.'}...
  [2] Tool: query_vendor_catalog({'fish_type': 'King Salmon'})
      Result: {'result': '[\n  {\n    "vendor": "Wild Salmon Co",\n    "fish_type": "King Salmon",\n    "available_lbs": 25,\n    "price_per_lb": 28,\n    "freshnes...
  [3] Tool: check_freshness({'vendor_name': 'Wild Salmon Co', 'fish_type': 'King Salmon'})
      Result: {'result': 'Wild Salmon Co King Salmon: freshness 9/10 (EXCELLENT). Suitable for sashimi: Yes'}...
  [4] Tool: estimate_shipping({'vendor_name': 'Wild Salmon Co', 'qty_lbs': 5})
      Result: {'result': '{"vendor": "Wild Salmon Co", "shipping_cost": 10.0, "delivery_days": 1, "method": "overnight cold pack"}'}...
  [5

---
## Phase 2: Negotiation & Human Approval

**Patterns: Agent-to-Agent Protocol (A2A) + AgentTool (Ch 7) + Human-in-the-Loop**

Now the system:
1. **Negotiator** proposes a price to the top vendor via A2A protocol
2. **Vendor** responds with accept/counter/reject
3. Up to 3 negotiation rounds
4. **GoalMonitor** checks if the deal meets budget/freshness/deadline
5. **Human Approval Gate** — customer confirms before purchase

```
[Negotiator] ──► OFFER: 5lb King Salmon @ $25/lb
    │
    ▼
[VendorAgent] ──► COUNTER: $26.50/lb, 5lb available
    │
    ▼
[Negotiator] ──► ACCEPT: $26.50/lb × 5lb = $132.50
    │
    ▼
[GoalMonitor] ──► Budget: ON_TRACK, Freshness: ON_TRACK
    │
    ▼
[Human] ──► "Approve $132.50 for 5lb King Salmon from Wild Salmon Co?" → YES
```

In [35]:
# ── Phase 2: Negotiation + Goals + Human Approval ───────

async def run_negotiation_and_approval(
    vendor_name: str, fish_type: str, qty: float,
    max_price_per_lb: float, goals: GoalTracker,
    auto_approve: bool = True  # set False for real human input
) -> Optional[Dict]:
    """Phase 2: Negotiate price, check goals, get human approval."""
    protocol = NegotiationProtocol()

    print(f"\n{'='*60}")
    print(f"NEGOTIATION: {qty}lb {fish_type} from {vendor_name}")
    print(f"Max price: ${max_price_per_lb}/lb (budget: ${goals.budget})")
    print(f"{'='*60}")

    # ── Round 1: Buyer offers below list price ───────────
    offer_price = max_price_per_lb * 0.85  # start 15% below max
    offer = protocol.create_offer("BuyerAgent", vendor_name,
                                   fish_type, qty, round(offer_price, 2))
    print(f"\n[BUYER] Offer: ${offer_price:.2f}/lb")

    # ── Vendor responds via MCP ───────────────────────
    server = mcp_servers[vendor_name]
    result = server.negotiate(fish_type, qty, offer_price)

    if result.get("accepted"):
        final_price = result["final_price"]
        protocol.accept(vendor_name, final_price, qty)
        print(f"[VENDOR] Accepted at ${final_price}/lb!")
    elif result.get("counter_price"):
        counter = result["counter_price"]
        protocol.create_counter(vendor_name, "BuyerAgent", counter, qty)
        print(f"[VENDOR] Counter-offer: ${counter}/lb")

        # ── Round 2: Buyer meets in the middle ──────────
        if counter <= max_price_per_lb:
            final_price = counter
            protocol.accept("BuyerAgent", final_price, qty)
            print(f"[BUYER] Accepted counter at ${final_price}/lb")
        else:
            # Try one more: split the difference
            split = round((offer_price + counter) / 2, 2)
            offer2 = protocol.create_offer("BuyerAgent", vendor_name,
                                            fish_type, qty, split)
            print(f"[BUYER] Counter-counter: ${split}/lb")
            result2 = server.negotiate(fish_type, qty, split)
            if result2.get("accepted") or (result2.get("counter_price") and result2["counter_price"] <= max_price_per_lb):
                final_price = result2.get("final_price", result2.get("counter_price"))
                protocol.accept("BuyerAgent", final_price, qty)
                print(f"[DEAL] Agreed at ${final_price}/lb")
            else:
                protocol.reject("BuyerAgent", "Cannot agree on price within budget")
                print(f"[NO DEAL] Cannot agree within budget")
                print(f"\n{protocol.trace()}")
                return None
    else:
        protocol.reject(vendor_name, result.get("reason", "Unknown"))
        print(f"[VENDOR] Rejected: {result.get('reason')}")
        print(f"\n{protocol.trace()}")
        return None

    total_cost = final_price * qty
    shipping = json.loads(estimate_shipping(vendor_name, qty))
    total_with_shipping = total_cost + shipping["shipping_cost"]

    # ── Goal Check ──────────────────────────────────
    freshness = VENDORS[vendor_name]["freshness_score"].get(fish_type, 5)
    print(f"\n{goals.report(total_with_shipping, freshness, shipping['delivery_days'])}")

    goal_status = goals.check(total_with_shipping, freshness, shipping["delivery_days"])
    if "VIOLATED" in goal_status.values():
        violated = [g for g, s in goal_status.items() if s == "VIOLATED"]
        print(f"\n❌ Goal violation: {', '.join(violated)}")
        return None

    # ── Human Approval Gate ─────────────────────────
    print(f"\n{'='*60}")
    print(f"PURCHASE SUMMARY:")
    print(f"  {qty}lb {fish_type} from {vendor_name}")
    print(f"  Price: ${final_price}/lb × {qty}lb = ${total_cost:.2f}")
    print(f"  Shipping: ${shipping['shipping_cost']:.2f} ({shipping['delivery_days']} days)")
    print(f"  Total: ${total_with_shipping:.2f}")
    print(f"  Freshness: {freshness}/10")
    print(f"{'='*60}")

    if auto_approve:
        print("\n✅ [HUMAN] Auto-approved (set auto_approve=False for manual)")
        approved = True
    else:
        response = input("Approve this purchase? (yes/no): ")
        approved = response.strip().lower() in ["yes", "y"]

    if not approved:
        print("❌ Purchase rejected by customer")
        return None

    # ── Record to Memory ────────────────────────────
    record_purchase(vendor_name, fish_type, qty, final_price, freshness)
    goals.spent += total_with_shipping

    deal = {
        "vendor": vendor_name, "fish_type": fish_type,
        "qty": qty, "price_per_lb": final_price,
        "total": total_with_shipping, "freshness": freshness,
        "delivery_days": shipping["delivery_days"],
    }

    print(f"\n{protocol.trace()}")
    print(f"\n✅ Purchase complete! Recorded to memory for future learning.")
    return deal


print("Negotiation pipeline ready: run_negotiation_and_approval()")

Negotiation pipeline ready: run_negotiation_and_approval()


In [36]:
# ── Run Phase 2: Negotiate with Wild Salmon Co ──────────

goals = GoalTracker(budget=150, min_freshness=8, delivery_by="Saturday")

deal = await run_negotiation_and_approval(
    vendor_name="Wild Salmon Co",
    fish_type="King Salmon",
    qty=5.0,
    max_price_per_lb=28.0,
    goals=goals,
    auto_approve=True,
)


NEGOTIATION: 5.0lb King Salmon from Wild Salmon Co
Max price: $28.0/lb (budget: $150)

[BUYER] Offer: $23.80/lb
[VENDOR] Counter-offer: $25.9/lb
[BUYER] Accepted counter at $25.9/lb

Goal Status (budget: $150, min freshness: 8, deliver by: Saturday):
  ⚠️ budget: AT_RISK
  ✅ freshness: ON_TRACK
  ✅ delivery: ON_TRACK

PURCHASE SUMMARY:
  5.0lb King Salmon from Wild Salmon Co
  Price: $25.9/lb × 5.0lb = $129.50
  Shipping: $10.00 (1 days)
  Total: $139.50
  Freshness: 9/10

✅ [HUMAN] Auto-approved (set auto_approve=False for manual)

Negotiation trace (3 rounds):
  Round 1: OFFER from BuyerAgent
    5.0lb King Salmon @ $23.8/lb
  Round 2: COUNTER_OFFER from Wild Salmon Co
    Counter: $25.9/lb, 5.0lb available
  Round 3: ACCEPTED from BuyerAgent
    Deal: 5.0lb @ $25.9/lb

✅ Purchase complete! Recorded to memory for future learning.


---
## Phase 3: Exception Handling & Recovery

**Patterns: Exception Handling + ReAct (Ch 8a) + Plan-and-Execute (Ch 8b)**

What happens when things go wrong? We inject faults and watch the system adapt:

| Fault | What Happens | Recovery Strategy |
|-------|-------------|------------------|
| `OUT_OF_STOCK` | Vendor has 0 inventory | Try next-best vendor |
| `PRICE_SPIKE` | Price jumped 50% | Negotiate harder or switch vendor |
| `VENDOR_OFFLINE` | MCP server unreachable | Skip, use backup vendor |

The **ExceptionHandler** logs every failure. The system retries with the next-best option from the discovery phase.

In [37]:
# ── Phase 3: Exception Handling with Fault Injection ─────

async def run_with_exceptions(
    fish_type: str, qty: float, max_price: float,
    budget: float, fault_vendor: str, fault_type: str
) -> Optional[Dict]:
    """Run the full pipeline with a fault injected at one vendor."""
    print(f"\n{'='*60}")
    print(f"SCENARIO: {fault_type} at {fault_vendor}")
    print(f"{'='*60}")

    # Inject fault into one vendor's MCP server
    original_server = mcp_servers[fault_vendor]
    mcp_servers[fault_vendor] = MockMCPVendorServer(
        fault_vendor, VENDORS[fault_vendor], fault_injection=fault_type)

    # Discover options (some will fail)
    print(f"\n[BUYER] Querying vendors for {fish_type}...")
    catalog_result = query_vendor_catalog(fish_type)
    options = json.loads(catalog_result) if catalog_result.startswith('[') else []

    if not options:
        print(f"  No available vendors! Exception log:")
        print(f"  {exceptions.summary()}")
        mcp_servers[fault_vendor] = original_server
        return None

    print(f"  Found {len(options)} available vendors:")
    for opt in options:
        print(f"    {opt['vendor']}: ${opt['price_per_lb']}/lb, "
              f"freshness {opt['freshness_score']}/10, {opt['available_lbs']}lb")

    # Try vendors in order until one works
    goals = GoalTracker(budget=budget, min_freshness=7, delivery_by="Saturday")
    deal = None

    for option in options:
        vendor = option["vendor"]
        print(f"\n[TRYING] {vendor}...")

        try:
            deal = await run_negotiation_and_approval(
                vendor_name=vendor, fish_type=fish_type,
                qty=qty, max_price_per_lb=max_price,
                goals=goals, auto_approve=True)

            if deal:
                print(f"\n✅ Success with {vendor}!")
                break
        except Exception as e:
            exc_msg = exceptions.record(type(e).__name__, vendor, str(e))
            print(f"  ❌ {exc_msg}")
            print(f"  Recovery: {exceptions.get_recovery(type(e).__name__)}")

    # Restore original server
    mcp_servers[fault_vendor] = original_server

    if not deal:
        print(f"\n❌ All vendors exhausted. Exception summary:")
        print(exceptions.summary())

    return deal


print("Exception handling pipeline ready")

Exception handling pipeline ready


In [38]:
# ── Scenario: Wild Salmon Co is OUT OF STOCK ───────────
# System should detect this and fall back to Atlantic Fishery

deal = await run_with_exceptions(
    fish_type="King Salmon", qty=5.0, max_price=28.0,
    budget=150.0,
    fault_vendor="Wild Salmon Co", fault_type="OUT_OF_STOCK"
)


SCENARIO: OUT_OF_STOCK at Wild Salmon Co

[BUYER] Querying vendors for King Salmon...
  Found 1 available vendors:
    Atlantic Fishery: $18/lb, freshness 6/10, 50lb

[TRYING] Atlantic Fishery...

NEGOTIATION: 5.0lb King Salmon from Atlantic Fishery
Max price: $28.0/lb (budget: $150.0)

[BUYER] Offer: $23.80/lb
[VENDOR] Accepted at $18/lb!

Goal Status (budget: $150.0, min freshness: 7, deliver by: Saturday):
  ✅ budget: ON_TRACK
  ❌ freshness: VIOLATED
  ✅ delivery: ON_TRACK

❌ Goal violation: freshness

❌ All vendors exhausted. Exception summary:
No exceptions recorded.


In [34]:
# ── Scenario: Price Spike at Wild Salmon Co ────────────
# King Salmon jumps from $28 to $42/lb. System should detect
# and try Atlantic Fishery instead.

deal2 = await run_with_exceptions(
    fish_type="King Salmon", qty=5.0, max_price=28.0,
    budget=150.0,
    fault_vendor="Wild Salmon Co", fault_type="PRICE_SPIKE"
)


SCENARIO: PRICE_SPIKE at Wild Salmon Co

[BUYER] Querying vendors for King Salmon...
  Found 2 available vendors:
    Wild Salmon Co: $42/lb, freshness 9/10, 25lb
    Atlantic Fishery: $18/lb, freshness 6/10, 50lb

[TRYING] Wild Salmon Co...

NEGOTIATION: 5.0lb King Salmon from Wild Salmon Co
Max price: $28.0/lb (budget: $150.0)

[BUYER] Offer: $23.80/lb
[VENDOR] Counter-offer: $25.9/lb
[BUYER] Accepted counter at $25.9/lb

Goal Status (budget: $150.0, min freshness: 7, deliver by: Saturday):
  ⚠️ budget: AT_RISK
  ✅ freshness: ON_TRACK
  ✅ delivery: ON_TRACK

PURCHASE SUMMARY:
  5.0lb King Salmon from Wild Salmon Co
  Price: $25.9/lb × 5.0lb = $129.50
  Shipping: $10.00 (1 days)
  Total: $139.50
  Freshness: 9/10

✅ [HUMAN] Auto-approved (set auto_approve=False for manual)

Negotiation trace (3 rounds):
  Round 1: OFFER from BuyerAgent
    5.0lb King Salmon @ $23.8/lb
  Round 2: COUNTER_OFFER from Wild Salmon Co
    Counter: $25.9/lb, 5.0lb available
  Round 3: ACCEPTED from BuyerAge

---
## Phase 4: Learning in Action

**Patterns: Memory Management + Learning & Adaptation**

Run the pipeline multiple times. Watch the system get smarter:
- Run 1: No history — buyer checks all vendors equally
- Run 2+: Learning engine recommends best vendor from past experience
- The `get_learning_insights` tool returns accumulated wisdom

In [39]:
# ── Learning Demo: Multiple Purchase Cycles ────────────

# Reset memory for clean demo
memory = PurchaseMemory()
learning = LearningEngine(memory)

print("=" * 60)
print("LEARNING DEMO: 3 purchase cycles for King Salmon")
print("=" * 60)

# Simulate 3 purchases from different vendors
purchases = [
    ("Wild Salmon Co", "King Salmon", 5, 28, 9, True),
    ("Atlantic Fishery", "King Salmon", 3, 18, 6, True),
    ("Wild Salmon Co", "King Salmon", 4, 27, 10, True),
]

for i, (vendor, fish, qty, price, fresh, on_time) in enumerate(purchases, 1):
    print(f"\n--- Purchase #{i} ---")
    memory.add_purchase(vendor, fish, qty, price, fresh, on_time)
    print(f"  Bought {qty}lb {fish} from {vendor} @ ${price}/lb (freshness: {fresh}/10)")
    
    # Check what the system has learned
    insights = learning.get_insights(fish)
    print(f"\n  {insights}")

print(f"\n{'='*60}")
print(f"FINAL MEMORY STATE:")
print(memory.summary())
print(f"\nBest vendor for King Salmon: {learning.best_vendor_for('King Salmon')}")
print(f"Average price paid: ${learning.avg_price('King Salmon'):.2f}/lb")

LEARNING DEMO: 3 purchase cycles for King Salmon

--- Purchase #1 ---
  Bought 5lb King Salmon from Wild Salmon Co @ $28/lb (freshness: 9/10)

  Learning insights for King Salmon:
  Past purchases: 1
  Best vendor: Wild Salmon Co
  Average price: $28.00/lb
  Best freshness seen: 9/10

--- Purchase #2 ---
  Bought 3lb King Salmon from Atlantic Fishery @ $18/lb (freshness: 6/10)

  Learning insights for King Salmon:
  Past purchases: 2
  Best vendor: Atlantic Fishery
  Average price: $23.00/lb
  Best freshness seen: 9/10

--- Purchase #3 ---
  Bought 4lb King Salmon from Wild Salmon Co @ $27/lb (freshness: 10/10)

  Learning insights for King Salmon:
  Past purchases: 3
  Best vendor: Wild Salmon Co
  Average price: $24.33/lb
  Best freshness seen: 10/10

FINAL MEMORY STATE:
Purchase history: 3 transactions
  Wild Salmon Co: 2 purchases, reliability 0.95
  Atlantic Fishery: 1 purchases, reliability 0.60

Best vendor for King Salmon: Wild Salmon Co
Average price paid: $24.33/lb


---
## Phase 5: Full Pipeline Integration

**All 14 patterns composed into one orchestrated flow.**

The `CapstoneOrchestrator` drives the entire pipeline:
1. Discovery (Routing + Parallel + Tool Use)
2. Quality inspection (Reflection)
3. Learning insights (Memory + Learning)
4. Negotiation (A2A Protocol)
5. Goal checking (Goal Monitoring)
6. Human approval (HITL)
7. Exception handling (Recovery on failure)
8. Record purchase (Memory update)

In [40]:
# ── Capstone Orchestrator ─────────────────────────────

class CapstoneOrchestrator:
    """Orchestrates the complete Pike Place fish-buying pipeline.
    
    Composes all 14 patterns into one deterministic flow.
    The orchestrator is Python (not an LLM) — same principle as Ch 8b.
    """

    def __init__(self):
        self.session_service = InMemorySessionService()
        self.buyer_runner = Runner(
            agent=buyer_agent, app_name="capstone",
            session_service=self.session_service)

    async def _call_buyer(self, message: str) -> str:
        """Send a message to the buyer agent."""
        session = await self.session_service.create_session(
            app_name="capstone", user_id="orchestrator")
        content = types.Content(role="user",
            parts=[types.Part.from_text(text=message)])
        response = ""
        async for event in self.buyer_runner.run_async(
            user_id="orchestrator", session_id=session.id, new_message=content):
            if event.is_final_response() and event.content:
                for part in event.content.parts:
                    if part.text:
                        response += part.text
        return response.strip()

    async def run(self, customer_request: str, budget: float,
                  min_freshness: int = 7, delivery_by: str = "Saturday",
                  fault_vendor: str = None, fault_type: str = None,
                  auto_approve: bool = True) -> Optional[Dict]:
        """Execute the complete fish-buying pipeline."""

        print(f"\n{'#'*60}")
        print(f"# CAPSTONE PIPELINE")
        print(f"# Request: {customer_request}")
        print(f"# Budget: ${budget}, Min Freshness: {min_freshness}/10")
        print(f"{'#'*60}")

        goals = GoalTracker(budget=budget, min_freshness=min_freshness,
                            delivery_by=delivery_by)

        # Inject fault if requested
        original_server = None
        if fault_vendor and fault_type:
            original_server = mcp_servers[fault_vendor]
            mcp_servers[fault_vendor] = MockMCPVendorServer(
                fault_vendor, VENDORS[fault_vendor], fault_injection=fault_type)
            print(f"\n⚠️  Fault injected: {fault_type} at {fault_vendor}")

        try:
            # ── Phase 1: Discovery via LLM Buyer Agent ───────
            print(f"\n┌─ Phase 1: DISCOVERY (Routing + Parallel + Tool Use)")
            buyer_result = await self._call_buyer(customer_request)
            print(f"│  Buyer found options")

            # ── Phase 2: Parse available vendors from catalog ──
            # (In parallel pipeline, we'd use ParallelAgent here)
            # For capstone, extract the fish type and query directly
            fish_type = None
            for fish in ["King Salmon", "Sockeye Salmon", "Pacific Halibut",
                         "Yellowfin Tuna", "Dungeness Crab"]:
                if fish.lower() in customer_request.lower():
                    fish_type = fish
                    break
            if not fish_type:
                fish_type = "King Salmon"  # default

            catalog = query_vendor_catalog(fish_type)
            options = json.loads(catalog) if catalog.startswith('[') else []

            if not options:
                print(f"└─ No vendors available for {fish_type}!")
                return None

            print(f"│  {len(options)} vendors available")

            # ── Phase 3: Learning insights ─────────────────
            print(f"│")
            print(f"├─ Phase 2: LEARNING INSIGHTS (Memory + Adaptation)")
            insights = learning.get_insights(fish_type)
            best_from_history = learning.best_vendor_for(fish_type)
            if best_from_history:
                print(f"│  Learning recommends: {best_from_history}")
                # Move preferred vendor to front
                options.sort(key=lambda x: (x['vendor'] != best_from_history,
                                            -x['freshness_score'], x['price_per_lb']))
            else:
                print(f"│  No history — first purchase of {fish_type}")

            # ── Phase 4: Try vendors with negotiation ───────
            print(f"│")
            print(f"├─ Phase 3: NEGOTIATION + GOALS + APPROVAL")

            # Extract qty from request (simple parse)
            qty = 5.0  # default
            for word in customer_request.split():
                try:
                    val = float(word)
                    if 0.5 <= val <= 50:
                        qty = val
                        break
                except ValueError:
                    continue

            deal = None
            for option in options:
                vendor = option["vendor"]
                max_price = min(option["price_per_lb"], budget / qty)

                print(f"│  Trying {vendor} (${option['price_per_lb']}/lb, "
                      f"freshness {option['freshness_score']}/10)...")

                # Quality gate (Reflection)
                if option["freshness_score"] < min_freshness:
                    print(f"│  ❌ Rejected by inspector: freshness {option['freshness_score']} < {min_freshness}")
                    exceptions.record("QUALITY_REJECTED", vendor,
                                      f"Freshness {option['freshness_score']} below minimum {min_freshness}")
                    continue

                try:
                    deal = await run_negotiation_and_approval(
                        vendor_name=vendor, fish_type=fish_type,
                        qty=qty, max_price_per_lb=max_price,
                        goals=goals, auto_approve=auto_approve)
                    if deal:
                        break
                except Exception as e:
                    exc_msg = exceptions.record(type(e).__name__, vendor, str(e))
                    print(f"│  ❌ {exc_msg}")

            # ── Final Summary ────────────────────────────
            print(f"│")
            if deal:
                print(f"└─ ✅ PIPELINE COMPLETE")
                print(f"   Purchased {deal['qty']}lb {deal['fish_type']} from {deal['vendor']}")
                print(f"   Total: ${deal['total']:.2f} (freshness: {deal['freshness']}/10)")
            else:
                print(f"└─ ❌ PIPELINE FAILED — no deal reached")
                print(f"   {exceptions.summary()}")

            return deal

        finally:
            # Restore original server
            if original_server:
                mcp_servers[fault_vendor] = original_server


print("CapstoneOrchestrator ready")

CapstoneOrchestrator ready


In [41]:
# ── Scenario 1: Happy Path (everything works) ─────────

# Reset for clean run
memory = PurchaseMemory()
learning = LearningEngine(memory)
exceptions = ExceptionHandler()

orchestrator = CapstoneOrchestrator()

deal1 = await orchestrator.run(
    customer_request=(
        "I need 5 lbs of fresh King Salmon for a dinner party this Saturday. "
        "Budget is $150. Needs to be sashimi-grade."
    ),
    budget=150,
    min_freshness=8,
)


############################################################
# CAPSTONE PIPELINE
# Request: I need 5 lbs of fresh King Salmon for a dinner party this Saturday. Budget is $150. Needs to be sashimi-grade.
# Budget: $150, Min Freshness: 8/10
############################################################

┌─ Phase 1: DISCOVERY (Routing + Parallel + Tool Use)
│  Buyer found options
│  2 vendors available
│
├─ Phase 2: LEARNING INSIGHTS (Memory + Adaptation)
│  No history — first purchase of King Salmon
│
├─ Phase 3: NEGOTIATION + GOALS + APPROVAL
│  Trying Wild Salmon Co ($28/lb, freshness 9/10)...

NEGOTIATION: 5.0lb King Salmon from Wild Salmon Co
Max price: $28/lb (budget: $150)

[BUYER] Offer: $23.80/lb
[VENDOR] Counter-offer: $25.9/lb
[BUYER] Accepted counter at $25.9/lb

Goal Status (budget: $150, min freshness: 8, deliver by: Saturday):
  ⚠️ budget: AT_RISK
  ✅ freshness: ON_TRACK
  ✅ delivery: ON_TRACK

PURCHASE SUMMARY:
  5.0lb King Salmon from Wild Salmon Co
  Price: $25.9/lb × 5.0

In [42]:
# ── Scenario 2: Exception Recovery (Wild Salmon out of stock) ─

deal2 = await orchestrator.run(
    customer_request="5 lbs King Salmon, budget $150",
    budget=150,
    min_freshness=5,  # relaxed for fallback
    fault_vendor="Wild Salmon Co",
    fault_type="OUT_OF_STOCK",
)


############################################################
# CAPSTONE PIPELINE
# Request: 5 lbs King Salmon, budget $150
# Budget: $150, Min Freshness: 5/10
############################################################

⚠️  Fault injected: OUT_OF_STOCK at Wild Salmon Co

┌─ Phase 1: DISCOVERY (Routing + Parallel + Tool Use)
│  Buyer found options
│  1 vendors available
│
├─ Phase 2: LEARNING INSIGHTS (Memory + Adaptation)
│  Learning recommends: Wild Salmon Co
│
├─ Phase 3: NEGOTIATION + GOALS + APPROVAL
│  Trying Atlantic Fishery ($18/lb, freshness 6/10)...

NEGOTIATION: 5.0lb King Salmon from Atlantic Fishery
Max price: $18/lb (budget: $150)

[BUYER] Offer: $15.30/lb
[VENDOR] Counter-offer: $16.65/lb
[BUYER] Accepted counter at $16.65/lb

Goal Status (budget: $150, min freshness: 5, deliver by: Saturday):
  ✅ budget: ON_TRACK
  ✅ freshness: ON_TRACK
  ✅ delivery: ON_TRACK

PURCHASE SUMMARY:
  5.0lb King Salmon from Atlantic Fishery
  Price: $16.65/lb × 5.0lb = $83.25
  Shipping: $1

In [43]:
# ── Scenario 3: Learning in Action (3rd purchase) ────────
# By now, memory has 2 purchases. The system should use
# learning insights to prefer the better vendor.

print("\n" + "=" * 60)
print("BEFORE RUN 3 — Learning State:")
print(memory.summary())
print(f"Best vendor for King Salmon: {learning.best_vendor_for('King Salmon')}")
print("=" * 60)

deal3 = await orchestrator.run(
    customer_request="5 lbs King Salmon, budget $150",
    budget=150,
    min_freshness=7,
)


BEFORE RUN 3 — Learning State:
Purchase history: 2 transactions
  Wild Salmon Co: 1 purchases, reliability 0.90
  Atlantic Fishery: 1 purchases, reliability 0.60
Best vendor for King Salmon: Atlantic Fishery

############################################################
# CAPSTONE PIPELINE
# Request: 5 lbs King Salmon, budget $150
# Budget: $150, Min Freshness: 7/10
############################################################

┌─ Phase 1: DISCOVERY (Routing + Parallel + Tool Use)
│  Buyer found options
│  2 vendors available
│
├─ Phase 2: LEARNING INSIGHTS (Memory + Adaptation)
│  Learning recommends: Atlantic Fishery
│
├─ Phase 3: NEGOTIATION + GOALS + APPROVAL
│  Trying Atlantic Fishery ($18/lb, freshness 6/10)...
│  ❌ Rejected by inspector: freshness 6 < 7
│  Trying Wild Salmon Co ($28/lb, freshness 9/10)...

NEGOTIATION: 5.0lb King Salmon from Wild Salmon Co
Max price: $28/lb (budget: $150)

[BUYER] Offer: $23.80/lb
[VENDOR] Counter-offer: $25.9/lb
[BUYER] Accepted counter at $25.9

In [44]:
# ── Final State: What the System Learned ───────────────

print("=" * 60)
print("FINAL SYSTEM STATE")
print("=" * 60)

print(f"\n{memory.summary()}")
print(f"\n{learning.get_insights('King Salmon')}")
print(f"\n{exceptions.summary()}")

FINAL SYSTEM STATE

Purchase history: 3 transactions
  Wild Salmon Co: 2 purchases, reliability 0.90
  Atlantic Fishery: 1 purchases, reliability 0.60

Learning insights for King Salmon:
  Past purchases: 3
  Best vendor: Atlantic Fishery
  Average price: $22.82/lb
  Best freshness seen: 9/10

Exception log: 1 events
  [QUALITY_REJECTED] Atlantic Fishery: Freshness 6 below minimum 7


---
## Key Takeaways

### All 14 Patterns in One System

| Pattern | Chapter | Where It Appeared |
|---------|---------|------------------|
| Routing | Ch 2 | Coordinator extracts fish type, routes to buyer specialist |
| Parallelization | Ch 3 | `query_vendor_catalog` checks all 5 vendors |
| Tool Use | Ch 5 | 7 tools: catalog, freshness, shipping, negotiate, learning, record, goals |
| Reflection | Ch 6 | QualityInspector rejects low-freshness options |
| Multi-Agent | Ch 7 | BuyerAgent, QualityInspector, GoalMonitor each with specialized roles |
| Sequential | Ch 7 | Discovery → Inspection → Negotiation → Approval → Record |
| AgentTool | Ch 7 | Negotiator uses vendor MCP as tool |
| ReAct | Ch 8a | BuyerAgent explores vendors iteratively with reasoning between calls |
| Plan-and-Execute | Ch 8b | Orchestrator replans when vendor fails (try next) |
| Memory | New | PurchaseMemory stores history across runs |
| Learning | New | LearningEngine finds best vendors from past data |
| MCP Server | New | MockMCPVendorServer simulates vendor tool endpoints |
| A2A Protocol | New | NegotiationProtocol: structured offer/counter/accept messages |
| Goal Tracking | New | GoalTracker: ON_TRACK / AT_RISK / VIOLATED for budget, freshness, deadline |
| Exception Handling | New | ExceptionHandler logs failures, suggests recovery strategies |
| Human-in-the-Loop | New | Approval gate before purchase (auto or manual) |

### Architecture Principles

| Principle | How We Applied It |
|-----------|------------------|
| **Separation of Concerns** | Each agent has one job (buy, inspect, negotiate, monitor) |
| **Deterministic Orchestration** | Python class drives the loop, not an LLM (Ch 8b lesson) |
| **Graceful Degradation** | Exception → log → try backup → escalate to human |
| **Explicit Protocols** | A2A negotiation, MCP tool calls, goal constraints — all structured |
| **Learnable State** | Memory persists across runs, improving vendor selection |
| **Transparent Reasoning** | Every agent call, tool use, and decision is logged and traceable |
| **Mock for Reproducibility** | Deterministic vendor data (Ch 6 lesson) — same results every run |
| **ADK Retry** | HttpRetryOptions on all agents (Ch 8b lesson) — no manual throttling |

### Notebook vs Production

| Component | This Notebook | Production System |
|-----------|--------------|------------------|
| Vendor Catalog | Mock dict with fixed data | Live vendor APIs or databases |
| MCP Server | In-process Python class | Actual vendor microservices |
| Learning | Simple best-of averages | ML recommendation engine |
| Goal Tracking | Python class with thresholds | Constraint solver (OR-Tools) |
| Human Approval | Auto-approve flag | Webhook + email/SMS notification |
| Memory | In-memory dict | PostgreSQL / Redis / data warehouse |
| Exception Handling | Fallback to next vendor | Retry + alerting + on-call escalation |

### The Core Lesson

**The pattern you wrap around an LLM determines how it thinks.**

Same Gemini 2.0 Flash model powers every agent in this notebook. But the Buyer reasons differently from the Inspector, which reasons differently from the Negotiator — because each is given a different architecture, different tools, and different constraints. Intelligence emerges from the system design, not just the model.